In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from PIL import Image
import time
import pandas as pd
import matplotlib.patches as patches

DATA_DIR = "/export/gaon1/data/jteneggi/data/malaria/"
df_training = pd.read_json("/export/gaon1/data/jteneggi/data/malaria/training.json")
df_test = pd.read_json("/export/gaon1/data/jteneggi/data/malaria/test_cropped.json")
frames = [df_training, df_test]
df_merged = pd.concat(frames, ignore_index=True)
# ADD IMAGE_NAME COLUMN TO DATAFRAME
image_names = []
for i, row in df_merged.iterrows():
    image_names.append(os.path.basename(row["image"]["pathname"]))
df_merged["image_name"] = image_names


def annotate(ax, image_name):
    query = df_merged.loc[df_merged["image_name"] == image_name]
    for i, row in query.iterrows():
        cells = row["objects"]
        for cell in cells:
            cell_class = cell["category"]
            if cell_class == "trophozoite":
                bbox = cell["bounding_box"]
                upper_left_r = bbox["minimum"]["r"]
                upper_left_c = bbox["minimum"]["c"]
                lower_right_r = bbox["maximum"]["r"]
                lower_right_c = bbox["maximum"]["c"]
                w = np.abs(lower_right_c - upper_left_c)
                h = np.abs(lower_right_r - upper_left_r)
                # Create a Rectangle patch
                rect = patches.Rectangle(
                    (upper_left_c, upper_left_r),
                    w,
                    h,
                    linewidth=2,
                    edgecolor="k",
                    facecolor="none",
                )
                # Add the patch to the Axes
                ax.add_patch(rect)


exp_mapper = ["hexp/absolute_0"]
examples = [
    "6c20abb2-c4bb-4de0-8774-03f4d30ce93d.png", 
    "9f38055f-4fe6-47fd-a6dd-c0743db4eccf.png", 
    "38d1b930-dd97-4248-9160-e9389a1a8dd7.png", 
    "59f5699f-885f-4004-b783-a60060144e43.png",
    "69b5b3de-ec64-4e5d-bcc2-44f495e06e76.png",
    "471f75c6-bd5d-4c20-bcc1-258e836f4d69.png"
]
for i, image_name in enumerate(examples):
    fig = plt.figure()
    ax = fig.subplots(1, 1)
    image_path = os.path.join(DATA_DIR, "trophozoite", "test", "1", image_name)
    image = Image.open(image_path)
    ax.imshow(image, alpha=0.5)
    ax.set_title("explanation")
    annotate(ax, image_name)
    for j, exp in enumerate(exp_mapper):
        explanation = np.load(os.path.join("true_positive_explanations", "%s/%s.npy" % (exp, image_name)))
        _abs = np.abs(explanation.flatten())
        _max = max(_abs)
        ax.imshow(explanation, cmap="bwr", alpha=0.5, vmax=_max, vmin=-_max)
        annotate(ax, image_name)
    plt.savefig("good_hshap_examples/%s.eps" % image_name)
    print("%d/%d saved figure" % (i + 1, len(examples)))
    plt.close()

1/6 saved figure
2/6 saved figure
3/6 saved figure
4/6 saved figure
5/6 saved figure
6/6 saved figure
